In [ ]:
#model3.pth 89.39%
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from efficientnet_pytorch import EfficientNet
import os
import json
import csv
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
from torch.optim.lr_scheduler import StepLR
from torch.cuda.amp import GradScaler, autocast  # 혼합 정밀도 학습

# 하이퍼파라미터 설정
batch_size = 32
epochs =300
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 경로 설정
train_dir = 'C:/Users/user/OneDrive/Desktop/Resnet182-real/data/category_data'
json_dir = 'C:/Users/user/OneDrive/Desktop/Resnet182-real/json'
csv_dir = 'C:/Users/user/OneDrive/Desktop/Resnet182-real/csv'
model_dir = 'C:/Users/user/OneDrive/Desktop/Resnet182-real/model'

# 필요한 디렉터리가 없으면 생성
os.makedirs(json_dir, exist_ok=True)
os.makedirs(csv_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

# 데이터 전처리 (크기 변경 및 정규화)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기를 224x224로 변경
    transforms.ToTensor(),  # Tensor로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
])

# 데이터셋 로드
dataset = datasets.ImageFolder(root=train_dir, transform=transform)

# 상위 폴더 이름을 기준으로 라벨을 지정
category_to_label = {folder: idx for idx, folder in enumerate(sorted(os.listdir(train_dir)))}

# 데이터 분할 (7:2:1 비율)
train_data, temp_data = train_test_split(dataset.samples, test_size=0.3, stratify=dataset.targets)
val_data, test_data = train_test_split(temp_data, test_size=0.33, stratify=[item[1] for item in temp_data])

# 데이터셋을 위한 커스텀 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, data, dataset, transform=None):
        self.data = data
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = self.dataset.loader(img_path)
        if self.transform:
            image = self.transform(image)
        return image, label

# 학습, 검증, 테스트 데이터로더 생성
train_dataset = CustomDataset(train_data, dataset, transform)
val_dataset = CustomDataset(val_data, dataset, transform)
test_dataset = CustomDataset(test_data, dataset, transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# EfficientNetB0 모델 정의
model = EfficientNet.from_pretrained('efficientnet-b0')  # 사전 학습된 EfficientNetB0 모델 불러오기
model._fc = nn.Linear(model._fc.in_features, len(dataset.classes))  # 마지막 레이어를 데이터셋 클래스 수에 맞게 수정
model = model.to(device)  # 모델을 CUDA 또는 CPU로 이동

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류를 위한 손실 함수
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Adam 옵티마이저 사용

# 학습률 스케줄러 설정
scheduler = StepLR(optimizer, step_size=10, gamma=0.7)  # 학습률을 10 에포크마다 0.7배로 감소

# 혼합 정밀도 학습 스케일러
scaler = GradScaler()

# 모델 성능을 평가하는 함수
def evaluate(model, dataloader):
    model.eval()  # 평가 모드로 설정
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():  # 평가 중에는 그래디언트 계산 안 함
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)  # 모델 출력
            loss = criterion(outputs, labels)  # 손실 계산
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)  # 예측값 계산
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  # 예측 정확도 계산
    accuracy = correct / total  # 정확도 계산
    avg_loss = running_loss / len(dataloader)  # 평균 손실값 계산
    return accuracy, avg_loss

# 학습 루프
best_val_acc = 0.0
best_epoch = 0
best_model_weights = None
epoch_log = []

# 얼리스탑 조건 (검증 정확도 99% 이상이면 학습 종료)
early_stop_threshold = 0.99

for epoch in range(epochs):
    model.train()  # 학습 모드로 설정
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(train_loader, desc=f"📢 Epoch [{epoch+1}/{epochs}] 시작"):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # 기울기 초기화
        with autocast():  # 혼합 정밀도
            outputs = model(inputs)  # 모델 출력
            loss = criterion(outputs, labels)  # 손실 계산
        scaler.scale(loss).backward()  # 역전파
        scaler.step(optimizer)  # 가중치 업데이트
        scaler.update()  # 스케일러 업데이트

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)  # 예측값 계산
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = correct / total  # 학습 정확도 계산
    train_loss = running_loss / len(train_loader)  # 학습 손실값 계산

    # 검증 데이터 평가
    val_acc, val_loss = evaluate(model, val_loader)

    # 최상의 검증 정확도를 기록한 모델 저장
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch + 1
        best_model_weights = model.state_dict()

    # 에포크 기록 저장
    epoch_log.append([epoch + 1, train_acc, train_loss, val_acc, val_loss])

    # 출력 형식 변경
    print(f"   🎯 Train Accuracy: {train_acc:.4f} | 📉 Train Loss: {train_loss:.4f} "
        f"| 🎯 Valid Accuracy: {val_acc:.4f} | 📉 Valid Loss: {val_loss:.4f}")

    # 얼리스탑 조건 체크 (검증 정확도가 0.99 이상이면 학습 종료)
    if val_acc >= early_stop_threshold:
        print(f"🚨 얼리스탑! 검증 정확도가 {val_acc:.4f}로 0.99에 도달하여 학습을 종료합니다.")
        break

    # 학습률 스케줄러 업데이트
    scheduler.step()

# 최상의 모델 저장
model.load_state_dict(best_model_weights)  # 최상의 모델 가중치로 업데이트
torch.save(model.state_dict(), os.path.join(model_dir, 'model3.pth'))  # 모델 저장

# 학습 기록을 CSV 파일로 저장
with open(os.path.join(csv_dir, '모델3기록.csv'), 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Epoch', 'Train Accuracy', 'Train Loss', 'Val Accuracy', 'Val Loss'])
    writer.writerows(epoch_log)

# 학습, 검증, 테스트 데이터셋 분할을 JSON 파일로 저장
splits = {
    "train": [{"image": item[0], "label": item[1]} for item in train_data],
    "val": [{"image": item[0], "label": item[1]} for item in val_data],
    "test": [{"image": item[0], "label": item[1]} for item in test_data]
}

# JSON 파일로 저장
with open(os.path.join(json_dir, 'model3train.json'), 'w') as f:
    json.dump(splits['train'], f, indent=4)

with open(os.path.join(json_dir, 'model3val.json'), 'w') as f:
    json.dump(splits['val'], f, indent=4)

with open(os.path.join(json_dir, 'model3test.json'), 'w') as f:
    json.dump(splits['test'], f, indent=4)

# 가장 좋은 검증 정확도를 기록한 에포크 출력
print(f"📢 Best Epoch: {best_epoch}, Best Validation Accuracy: {best_val_acc:.4f}")

# 최종 모델을 테스트 데이터로 평가
test_acc, test_loss = evaluate(model, test_loader)
print(f"🎯 Test Accuracy: {test_acc:.4f}, 📉 Test Loss: {test_loss:.4f}")


C:\Users\user\AppData\Local\Temp\ipykernel_10236\2041518279.py:89: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Loaded pretrained weights for efficientnet-b0


📢 Epoch [1/300] 시작:   0%|          | 0/304 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_10236\2041518279.py:128: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # 혼합 정밀도
📢 Epoch [1/300] 시작: 100%|██████████| 304/304 [02:24<00:00,  2.10it/s]


   🎯 Train Accuracy: 0.7420 | 📉 Train Loss: 0.8001 | 🎯 Valid Accuracy: 0.7502 | 📉 Valid Loss: 0.8920


📢 Epoch [2/300] 시작: 100%|██████████| 304/304 [02:29<00:00,  2.03it/s]


...

   🎯 Train Accuracy: 1.0000 | 📉 Train Loss: 0.0001 | 🎯 Valid Accuracy: 0.8932 | 📉 Valid Loss: 0.7144


📢 Epoch [299/300] 시작: 100%|██████████| 304/304 [02:15<00:00,  2.25it/s]


   🎯 Train Accuracy: 0.9998 | 📉 Train Loss: 0.0005 | 🎯 Valid Accuracy: 0.8918 | 📉 Valid Loss: 0.7131


📢 Epoch [300/300] 시작: 100%|██████████| 304/304 [02:15<00:00,  2.25it/s]


   🎯 Train Accuracy: 0.9999 | 📉 Train Loss: 0.0002 | 🎯 Valid Accuracy: 0.8921 | 📉 Valid Loss: 0.7161
📢 Best Epoch: 208, Best Validation Accuracy: 0.8939
🎯 Test Accuracy: 0.8916, 📉 Test Loss: 0.7997


In [13]:
import torch
import json
from torchvision import transforms
from PIL import Image

# 모델 로드
model = EfficientNet.from_pretrained('efficientnet-b0')
model._fc = nn.Linear(model._fc.in_features, len(dataset.classes))
model.load_state_dict(torch.load('C:/Users/user/OneDrive/Desktop/Resnet182-real/model/model3.pth'))  # 모델 경로 수정
model = model.to(device)
model.eval()  # 평가 모드로 설정

# 이미지 전처리 (train 때와 동일)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# JSON 파일 로드
with open('C:/Users/user/OneDrive/Desktop/Resnet182-real/json/model3train.json', 'r') as f:
    train_data = json.load(f)
with open('C:/Users/user/OneDrive/Desktop/Resnet182-real/json/model3val.json', 'r') as f:
    val_data = json.load(f)
with open('C:/Users/user/OneDrive/Desktop/Resnet182-real/json/model3test.json', 'r') as f:
    test_data = json.load(f)

# 정확도 계산 함수 (이미지 별 결과 출력)
def evaluate_model_on_json_data(data):
    correct = 0
    total = 0
    results = []  # 각 이미지에 대한 결과 저장
    with torch.no_grad():
        for item in data:
            img_path = item['image']
            label = item['label']
            
            # 이미지 열기 및 전처리
            image = Image.open(img_path).convert('RGB')
            image = transform(image).unsqueeze(0).to(device)  # 배치 차원 추가 후 device로 이동
            
            # 모델 예측
            output = model(image)
            _, predicted = torch.max(output, 1)
            
            # 결과 기록 (맞추었는지 틀렸는지)
            result = "✅ 맞춤" if predicted.item() == label else "❌ 틀림"
            results.append(f"이미지: {img_path} | 예측: {predicted.item()} | 실제: {label} | 결과: {result}")

            # 예측이 맞으면 correct 카운트 증가
            if predicted.item() == label:
                correct += 1
            total += 1

    return correct, total, results

# Train 데이터셋 평가
train_correct, train_total, train_results = evaluate_model_on_json_data(train_data)
print(f"Train Accuracy: {train_correct}/{train_total} = {train_correct / train_total:.4f}")
print("\n🎯 Train 데이터셋 결과:")
for result in train_results:
    print(result)

# Validation 데이터셋 평가
val_correct, val_total, val_results = evaluate_model_on_json_data(val_data)
print(f"\nValidation Accuracy: {val_correct}/{val_total} = {val_correct / val_total:.4f}")
print("\n🎯 Validation 데이터셋 결과:")
for result in val_results:
    print(result)

# Test 데이터셋 평가
test_correct, test_total, test_results = evaluate_model_on_json_data(test_data)
print(f"\nTest Accuracy: {test_correct}/{test_total} = {test_correct / test_total:.4f}")
print("\n🎯 Test 데이터셋 결과:")
for result in test_results:
    print(result)


Loaded pretrained weights for efficientnet-b0
Train Accuracy: 9717/9718 = 0.9999

🎯 Train 데이터셋 결과:
이미지: C:/Users/user/OneDrive/Desktop/Resnet182-real/data/category_data\매트·침낭\자충매트\google_캠핑 자충매트_71.jpg | 예측: 4 | 실제: 4 | 결과: ✅ 맞춤
이미지: C:/Users/user/OneDrive/Desktop/Resnet182-real/data/category_data\매트·침낭\에어매트\naver_캠핑 에어매트_131.jpg | 예측: 4 | 실제: 4 | 결과: ✅ 맞춤
이미지: C:/Users/user/OneDrive/Desktop/Resnet182-real/data/category_data\화로대·버너\화로대\google_캠핑 화로대_123.jpg | 예측: 9 | 실제: 9 | 결과: ✅ 맞춤
이미지: C:/Users/user/OneDrive/Desktop/Resnet182-real/data/category_data\텐트·타프\텐트\naver_캠핑 텐트_327.jpg | 예측: 8 | 실제: 8 | 결과: ✅ 맞춤
이미지: C:/Users/user/OneDrive/Desktop/Resnet182-real/data/category_data\매트·침낭\침낭\naver_캠핑 침낭_210.jpg | 예측: 4 | 실제: 4 | 결과: ✅ 맞춤
이미지: C:/Users/user/OneDrive/Desktop/Resnet182-real/data/category_data\냉·난방용품\난로·히터\naver_캠핑 난로·히터_227.jpg | 예측: 2 | 실제: 2 | 결과: ✅ 맞춤
이미지: C:/Users/user/OneDrive/Desktop/Resnet182-real/data/category_data\냉·난방용품\난로·히터\google_캠핑 난로·히터_64.jpg | 예측: 2 | 실제: 2 | 결과

In [3]:
import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet
import os
import json
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder

# 하이퍼파라미터 설정
batch_size = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 경로 설정
train_dir = 'C:/Users/user/OneDrive/Desktop/Resnet182-real/data/category_data'
json_dir = 'C:/Users/user/OneDrive/Desktop/Resnet182-real/json'
model_dir = 'C:/Users/user/OneDrive/Desktop/Resnet182-real/model'

# 데이터 전처리 (크기 변경 및 정규화)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기를 224x224로 변경
    transforms.ToTensor(),  # Tensor로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
])

# 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]['image']
        label = self.data[idx]['label']
        image = datasets.folder.default_loader(img_path)
        if self.transform:
            image = self.transform(image)
        return image, label

# 학습한 모델 불러오기
model = EfficientNet.from_pretrained('efficientnet-b0')  # 사전 학습된 EfficientNetB0 모델 불러오기
model._fc = nn.Linear(model._fc.in_features, len(os.listdir(train_dir)))  # 마지막 레이어 수정
model.load_state_dict(torch.load(os.path.join(model_dir, 'model3.pth')))  # 저장된 가중치 파일 불러오기
model = model.to(device)
model.eval()  # 평가 모드로 설정

# 평가 함수 정의
def evaluate(model, dataloader):
    model.eval()  # 평가 모드로 설정
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():  # 평가 중에는 그래디언트 계산 안 함
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)  # 모델 출력
            loss = nn.CrossEntropyLoss()(outputs, labels)  # 손실 계산
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)  # 예측값 계산
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  # 예측 정확도 계산
    accuracy = correct / total  # 정확도 계산
    avg_loss = running_loss / len(dataloader)  # 평균 손실값 계산
    return accuracy, avg_loss

# JSON 파일로부터 데이터셋 불러오기
with open(os.path.join(json_dir, 'model3train.json'), 'r') as f:
    train_data = json.load(f)

with open(os.path.join(json_dir, 'model3val.json'), 'r') as f:
    val_data = json.load(f)

with open(os.path.join(json_dir, 'model3test.json'), 'r') as f:
    test_data = json.load(f)

# 라벨 인코딩
label_encoder = LabelEncoder()
label_encoder.fit([item['label'] for item in train_data])  # 학습 데이터에서 라벨 추출

# 데이터로더 생성
train_loader = DataLoader(CustomDataset(train_data, transform), batch_size=batch_size, shuffle=False)
val_loader = DataLoader(CustomDataset(val_data, transform), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(CustomDataset(test_data, transform), batch_size=batch_size, shuffle=False)

# 학습, 검증, 테스트 데이터셋 정확도 평가
train_acc, train_loss = evaluate(model, train_loader)
val_acc, val_loss = evaluate(model, val_loader)
test_acc, test_loss = evaluate(model, test_loader)

# 출력
print(f"🎯 Train Accuracy: {train_acc:.4f}, 📉 Train Loss: {train_loss:.4f}")
print(f"🎯 Validation Accuracy: {val_acc:.4f}, 📉 Validation Loss: {val_loss:.4f}")
print(f"🎯 Test Accuracy: {test_acc:.4f}, 📉 Test Loss: {test_loss:.4f}")


Loaded pretrained weights for efficientnet-b0
🎯 Train Accuracy: 0.9999, 📉 Train Loss: 0.0002
🎯 Validation Accuracy: 0.8921, 📉 Validation Loss: 0.7161
🎯 Test Accuracy: 0.8916, 📉 Test Loss: 0.7997
